# Searcher类实现搜索算法

In [1]:
import pickle
import numpy as np

In [17]:
class Searcher(object):
    def __init__(self):
        with open('./var/poem_list', 'rb') as f:
            self.poem_list = pickle.load(f)
        # 按诗人的倒排索引
        with open('./var/poet2poem', 'rb') as f:
            self.poet2poem = pickle.load(f)
        self.poem_num = len(self.poem_list)
        self.results = []
        self.keywords = []
        self.TF_IDF = np.load('./var/TF_IDF.npz')['arr']
        with open('./var/word2id', 'rb') as f:
            self.word2id = pickle.load(f)
        with open('./var/id2word', 'rb') as f:
            self.id2word = pickle.load(f)
        with open('./var/synonym_list', 'rb') as f:
            self.synonym_list = pickle.load(f)
        with open('./var/big2id', 'rb') as f:
            self.big2id = pickle.load(f)
        with open('./var/id2big', 'rb') as f:
            self.id2big = pickle.load(f)
    
    def new_search(self, poet, keywords):
        '''
        发起新的搜索
        '''
        # 根据诗人搜素
        poet_lim = []
        if poet == '':
            poet_lim = list(range(self.poem_num))
        elif poet not in self.poet2poem:
            self.results.append([])
            self.keywords.append([])
            return
        else:
            poet_lim = []
            for i in self.poet2poem[poet]:
                poet_lim.append(i)
        poet_dict = {}
        for i in poet_lim:
            poet_dict[i] = 1

        # 根据关键词搜索
        if keywords.split() == []:
            res = []
            for i in poet_lim:
                res.append(self.poem_list[i])
            self.results.append(res)
            self.keywords.append([])
            return
            
        # 为每一首诗打分
        kw = keywords.split()
        append_kw = kw.copy()
        
        score = np.zeros(self.poem_num)
        for word in kw:
            word_id = self.word2id[word]
            score += self.TF_IDF[:, word_id] * 1000
            if word_id in self.id2big:
                word_big = self.id2big[word_id]
                factor = 1
                for syn_big in self.synonym_list[word_big]:
                    syn_id = self.big2id[syn_big]
                    append_kw.append(self.id2word[syn_id])
                    score += self.TF_IDF[:, syn_id] * factor
                    factor /= 2
            
        
        res_ind = np.argsort(-score)
        res_ind = res_ind[score[res_ind] > 1e-6]
        res_ind = res_ind[[(x in poet_dict) for x in res_ind]]
        
        if res_ind.shape[0] <= 0:
            self.results.append([])
            self.keywords.append(append_kw)
            return
        
        #print('result number:', len(res_ind))
        if res_ind.shape[0] > 100:
            res_ind = res_ind[:100]
        
        # HITS
        edge = self.make_edge(res_ind, self.TF_IDF)
        aut = self.HITS(edge)
        
        pairs = sorted(list(zip(aut, res_ind)))
        pairs.reverse()
        res_ind = [i[1] for i in pairs]
        
        # 重整，含有 keyword 的提上去
        res_kw = []
        res_syn = []
        for poem in res_ind:
            has_kw = False
            for word in kw:
                if self.TF_IDF[poem, self.word2id[word]] > 0:
                    has_kw = True
                    break
            if has_kw:
                res_kw.append(poem)
            else:
                res_syn.append(poem)
        res_ind = res_kw + res_syn
        
        res = []
        for i in res_ind:
            res.append(self.poem_list[i])
        self.results.append(res)
        self.keywords.append(append_kw)
            
    def make_edge(self, search_res, TF_IDF):
        '''
        根据搜索结果建超链接图
        '''
        search_res = np.sort(search_res)

        feature = TF_IDF[search_res]
        #print(search_res)

        size = len(search_res)

        from sklearn.decomposition import PCA
        dec_to = min(32, feature.shape[0], feature.shape[1])
        feature = PCA(n_components = dec_to).fit_transform(feature)

        edge = np.zeros((size, size))

        for i in range(size):
            for j in range(size):
                edge[i][j] = edge[j][i] = np.linalg.norm(feature[i]-feature[j])

        # 正常的网页中，超链接数和网页数应当是同阶的
        # 这里，就选最大的 5n 条边
        threshold = np.sort(edge.flatten())[min(5 * size, size * size - 1)]
        for i in range(size):
            for j in range(size):
                edge[i, j] = 1 if edge[i, j] < threshold else 0
        return edge
    
    def HITS(self, edge):
        '''
        HITS 算法，读入代表超链接的图，
        返回每个节点的权威值
        '''
        max_iteration = 200
        error = 0.0001
        size = edge.shape[0]

        aut = np.ones(size).T
        hub = np.ones(size).T

        k = 0
        while k < max_iteration:
            err = 0
            k += 1

            new_aut = edge.T @ hub
            new_hub = edge @ new_aut

            new_aut /= np.linalg.norm(new_aut)
            new_hub /= np.linalg.norm(new_hub)

            if np.linalg.norm(new_aut - aut) < error:
                break

            aut = new_aut
            hub = new_hub

        return aut.T

In [18]:
# 以下是测试
s = Searcher()
s.new_search('李白', '西风')

In [19]:
s.keywords[0]

['西风', '北风', '狂风', '凉风']

In [20]:
s.results[0]

[]

In [6]:
s.poem_list[372]

'紫藤树\n李白\n紫藤挂云木\n花蔓宜阳春\n密叶隐歌鸟\n香风留美人\n'

In [7]:
s.poem_list[381]

'春怨\n李白\n白马金羁辽海东\n罗帷绣被卧春风\n落月低轩窥烛尽\n飞花入户笑床空\n'

In [8]:
s.word2id['春']

2059